In [2]:
# Group: 28
# Authors: Thieme Brandwagt and Sam Buisman
#=========================================
# Assignment 2 - Aircraft Routing Problem 
#=========================================

# Import necessary packages
from openpyxl import *
from time import *
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# profit, n, aircraft, i, j = s # Attributes for state


### STEP 1: Load the data

In [4]:
# Load airport data into a numpy array
file_path_airports = "DATA/AirportData.xlsx"
airports_df = pd.read_excel(file_path_airports, usecols ="B:U")
airports = airports_df.to_numpy()
print(airports[0][0])
# Create list for all the airports (columns)
cities = airports_df.columns
print(cities[0])


# Load fleet data into a numpy array
file_path_fleet = "DATA/FleetType.xlsx"
fleet_df = pd.read_excel(file_path_fleet)
fleet_df = fleet_df.set_index("Aircraft Type")
fleet = fleet_df.to_numpy()
print(fleet[0][0])

# Create list for all the aircraft types (columns)
aircraft_types = fleet_df.columns
print(aircraft_types[0])


# Load demand data into a numpy array
file_path_demand = "DATA/Group28.xlsx"
demand_df = pd.read_excel(file_path_demand, usecols="B:AG", skiprows=4)
demand = demand_df.to_numpy()
print(demand[0][0])

# Create a triple dimensional array for demand data (origin x destination x time)
demand_matrix = np.zeros((len(cities), len(cities), demand.shape[1] - 2))
# Populate matrix with all demand from all intervals
for rows in range(len(cities)):
    for columns in range(len(cities)):
        for time_steps in range(int(120/4)):  # 120 hours devided by 4 hour time steps
            demand_matrix[rows][columns][time_steps] = demand[rows * 20 + columns][time_steps + 2]
print(demand_matrix[19][18][6])


LHR
London
800.0
Type 1: Small freighter
LHR
11396.314893372413


In [5]:
time_intervals = ["00:00-04:00", "04:00-08:00", "08:00-12:00", 
                  "12:00-16:00", "16:00-20:00", "20:00-00:00"]

In [6]:
tijd = [0,1,2,3,4,5]
tijd[-1]

5

### STEP 2: Define the functions

In [7]:
# Adding the previous two bins to the currect bin to get total demand
def flow(i, j, t):
    return demand_matrix[i,j,t] + 0.2 * (demand_matrix[i,j,t-1]+demand_matrix[i,j,t-2])

print(flow(0,1,25)) # LHR to CDG for bin 8

51127.0017609541


In [8]:
def distance(i, j, R_E=6371):
    if i != j:
        lat_i, long_i = airports[1][i], airports[2][i]
        lat_j, long_j = airports[1][j], airports[2][j]
        delta_lat = lat_j - lat_i
        delta_long = long_j - long_i
        root = math.sin(math.radians(delta_lat)/2)**2 + math.cos(math.radians(lat_i)) \
            * math.cos(math.radians(lat_j)) * math.sin(math.radians(delta_long)/2)**2
        return R_E * 2 * math.asin(math.sqrt(root)) 
    else:
        return 0

# E.v.t. nog stoppen in een df'pje later, zodat ie niet naar zichzelf gaat verwijzen

In [9]:
time_step = 6
total_time = 120 * 60
# print(stages)
# print(stages[5])

def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k]

time_stages = np.arange(0, total_time, time_step)
time_bins = np.arange(30)

divider = len(time_stages)/ len(time_bins) # Divide by 40 to get the right bin associated with the stage (round down)

print(len(time_stages))
print(divider)

1200
40.0


In [20]:
# Calculate the profit for each flight leg
def cost_flight_leg(i, j, k):
    if i != j:
        tc = fleet[7][k] * (distance(i, j, R_E=6371) / fleet[0][k])     # formula for the time costs
        fc = ((fleet[8][k] * 1.42) / 1.5) * distance(i, j, R_E=6371)    # formula for the fuel costs
        foc = fleet[6][k]                                               # formula for the fixed operating costs
        return tc + fc + foc

print(cost_flight_leg(0,1,0))

# Calculate the profit for each flight leg
def profit(i, j, k, n):
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    if i != j:
        revenue = 0.26 * distance(i,j) * satisfied_demand / 1000
        costs = cost_flight_leg(i,j,k)
        return revenue - costs
    else:
        return 0
print(flow(0,1,26))
print(profit(0, 1, 0, 1050))
print(fleet[1][0])

# Calculate the flight duration for each flight leg
# ONDERSTAANDE TWEE FUNCTIES KUNNEN LATER EVT GEMERGED WORDEN
def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k] * 60 + 15 + 15 + fleet[2][k] # Round up to the nearest 6-fold
print(flight_duration(1,2,1)) # From Paris to Amsterdam

def get_stages_in_flight(i,j,k):
    return math.ceil((flight_duration(i,j,k)) / 6)

print(get_stages_in_flight(1,2,1))

2385.302572558103
8403.575400678272
-1626.7664906330197
23000.0
178.1131589151272
30


In [11]:
# # DEZE CEL KAN LATER MISSCHIEN WEG, MAAR WE ZOUDEN DE STATUS KUNNEN BEPALEN DOOR DUS ALTIJD EEN I EN J MEE TE GEVEN AAN EEN VLIEGTUIG, DUS I NAAR I ALS HET VLIEGTUIG STILSTAAT OP DE HUB EN J NAAR J OP EEN SPOKE, I NAAR J ALS IE ONDERWEG IS
# state = ['grounded_hub', 
#           'grounded_spoke', 
#           'airborne_to_hub', 
#           'airborne_to_spoke'] 

# # Determine current state (what is the current status of the plane?)
# def get_current_state(i, j):
#     if i == j:  # grounded
#         return state[0] if i == 3 else state[1] # grounded at hub or spoke
#     else:  
#         return state[2] if j == 3 else state[3] # airborne to hub or to spoke

# get_current_state(4,4)

In [21]:
# Create a list of all possible decisions a plane can take at airport i at departure stage n
def get_candidates(i,k,n):    
    candidates = []
    hub = 3 # Define Frankfurt as a hub
    if i == hub:                 
        for j in range(len(cities)):                                                        
            if i == j:
                arrival_stage = n+1
                if arrival_stage < 1200:                                                     # make sure no airplane is flying after the fifth day
                    candidates.append([n, arrival_stage, i, j, profit(i,i,k,n)])             # grounded at hub
            else:
                arrival_stage = n+get_stages_in_flight(i,j,k)
                if arrival_stage < 1200:
                    candidates.append([n, arrival_stage, i, j, profit(i,j,k,arrival_stage)]) # flight to spoke
        
    else:
        if n+1 < 1200:
            candidates.append([n, n+1 ,i,i, profit(i,i,k,n)])   
        arrival_stage = n+get_stages_in_flight(i,hub,k)                               # grounded at spoke
        if arrival_stage < 1200:
            candidates.append([n, arrival_stage, i, hub, profit(i,hub,k,arrival_stage)])     # flight to hub
    return candidates

get_candidates(3,0,1190)

[[1190, 1191, 3, 3, 0]]

##### ^ [departure_stage, arrival_stage, departure_airport, arrival_airport, profit] ^

## Implementation of a memory

In [28]:
memo = {}

def get_routes(departure_airport, departure_stage):
    # Check if the result is already cached
    if (departure_airport, departure_stage) in memo:
        return memo[(departure_airport, departure_stage)]
    
    routes = []
    
    for c in get_candidates(departure_airport, 0, departure_stage):
        arrival_stage = c[1]
        arrival_airport = c[3]
        future_routes = get_routes(arrival_airport, arrival_stage)
        
        if future_routes:
            for route in future_routes:
                if distance(c[2],c[3]) <= fleet[3][0] and airports[3][c[2]] >= fleet[4][0] and airports[3][c[3]] >= fleet[4][0]:
                    total_profit = c[4] + route[0]  # Add current profit to future route's profit
                    routes.append([total_profit, c[2]] + route[1:])  # Add the total profit to the route
        else:
            total_profit = c[4]  # Profit for the last segment
            routes.append([total_profit, c[2], c[3]])  # Append the total profit and segment profit
    
    # Only keep itineraries that have Frankfurt as the last node visited
    maskdoorbas = [(route[-1] == 3) for route in routes]

    sliced_routes = []
    for i in range(len(routes)):
        if maskdoorbas[i]:
            sliced_routes.append(routes[i])
    # Only keep the route with the highest profit for each departure_airport
 
    sliced_routes = sorted(sliced_routes, key=lambda route: route[0], reverse=True)

    if sliced_routes:
        max_profit_route = sliced_routes[0]  # Take the route with the highest profit
        memo[(departure_airport, departure_stage)] = [max_profit_route]  # Cache only the highest profit route for the current stage
    else:
        memo[(departure_airport, departure_stage)] = []
    
    return memo[(departure_airport, departure_stage)]

# def get_state(departure_airport, departure_stage):
#     return max(get_routes(departure_airport, departure_stage), key=lambda route: route[0]) if get_routes(departure_airport, departure_stage) else []

# Clear the memo dictionary before running to avoid stale data
memo.clear()

# Example call
print(get_routes(3, 0))



[[13690.753708397682, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,

##### ^ [total_profit, departure_airport, via, ... , via, arrival_airport] ^